# Promises 
## are about retrieveing or calculating something asynchroniously. 
### Before promises this was done using callbacks - functions called when result is ready.
*Callbacks can be called **on any event**, but this is out of our current scope.*

## Classic callback

In [4]:
function addExtraC(price, callback) {
    setTimeout(function() {
        callback(price + 1);
    }, 300);
}

addExtraC(1, newPrice => {
    if (newPrice > 2) {
        console.log("Price cannot exceed 3");
    }
    else {
        console.log(newPrice);
    }
});

undefined

2


## Same with promise

In [2]:
function addExtraP(price) {
    return new Promise(function(resolve, reject) {
        if (price > 2)
            reject("Price cannot exceed 3");
        setTimeout(function() {
            resolve(price + 1);
        }, 300);
    });
}

addExtraP(1).then(newPrice => console.log(newPrice));

2


undefined

### a promise is definitely better than callback when we have 
## a chain of asynchronious actions
### each to be done basing on the result of previous one. 

In [6]:
addExtraP(1).then(addExtraP).then(addExtraP).then(newPrice => console.log(newPrice), error => console.log(error));

Price cannot exceed 3


undefined

### The same using catch:

In [7]:
addExtraP(1).then(addExtraP).then(addExtraP).then(newPrice => console.log(newPrice)).catch(error => console.log(error));

Price cannot exceed 3


undefined

### error handling can be mentioned once - at the end - and called when really needed
#### (error propagation)

In [3]:
addExtraP(1).then(newPrice => {console.log(newPrice); return addExtraP(newPrice);})
            .then(newPrice => {console.log(newPrice); return addExtraP(newPrice);})
            .then(newPrice => {console.log(newPrice); return addExtraP(newPrice);})
            .then(newPrice => {console.log(newPrice); return addExtraP(newPrice);}, error => console.log(error));

2
3
Price cannot exceed 3


undefined

## Just to retrieve a value (document) asyncroniously

In [10]:
let v0 = new Promise(function(resolve, reject) {
    setTimeout(function() {resolve(99.9);}, 300);
});

v0.then(res => console.log(res));

99.9


undefined

### If we need a number of results and can do something when receive them all and only in case they all have sucseeded. 
## Promise.all()
#### Besides promises we can add values or functions returning values:

In [1]:
function greet() {
    return "Hello!";
}

let vals = [200
          , new Promise((resolve, reject) => setTimeout(() => resolve(99.9), 300))
          , new Promise((resolve, reject) => setTimeout(() => resolve(199.9), 800))
          , greet()];

Promise.all(vals).then(solved => console.log(solved));

[ 200, 99.9, 199.9, 'Hello!' ]


undefined

In [1]:
let rej = [new Promise((resolve, reject) => setTimeout(() => resolve(100), 300))
         , new Promise((resolve, reject) => setTimeout(() => reject("An error occured on slow one"), 800))];

Promise.all(rej).then(solved => console.log(solved)).catch(error => console.log(error));

An error occured on slow one


undefined

In [2]:
let rejall = [new Promise((resolve, reject) => setTimeout(() => reject("An error occured on fast one"), 300))
            , new Promise((resolve, reject) => setTimeout(() => reject("An error occured on slow one"), 800))];

Promise.all(rejall).then(solved => console.log(solved)).catch(error => console.log(error));

An error occured on fast one


undefined

### If we need a number of results and will do something with each resolved result when all are resolved or rejected
## Promise.allSettled()

In [4]:
let settled = [new Promise((resolve, reject) => setTimeout(() => resolve(100), 300))
             , new Promise((resolve, reject) => setTimeout(() => reject("An error occured on slow one"), 800))];

Promise.allSettled(settled).then(solved => console.log(solved)).catch(error => console.log(error));

[
  { status: 'fulfilled', value: 100 },
  { status: 'rejected', reason: 'An error occured on slow one' }
]


undefined

### If we can receive a data from a number of sources, we can create promises fro all of them and then race - the result of the fastest one will be returned.
## Promise.race()

In [3]:
let fast = new Promise((resolve, reject) => setTimeout(() => resolve("fast"), 100));
let slow = new Promise((resolve, reject) => setTimeout(() => resolve("slow"), 200));

Promise.race([fast, slow]).then(res => console.log(res)).catch(error => console.log(error));

fast


undefined

### But if fastest promise is rejected - we will not receive any result:

In [4]:
let fast1 = new Promise((resolve, reject) => setTimeout(() => reject("fast is rejected"), 100));
let slow1 = new Promise((resolve, reject) => setTimeout(() => resolve("slow"), 200));

Promise.race([fast1, slow1]).then(res => console.log(res)).catch(error => console.log(error));

fast is rejected


undefined

### More often we'd like to retrieve the result if it can be retrieved from any of possible sources.
## Promise.any()

In [1]:
const sources = [Promise.reject(0)
               , new Promise((resolve) => setTimeout(resolve, 100, "quick"))
               , new Promise((resolve) => setTimeout(resolve, 300, "slow"))];

Promise.any(sources).then((val) => console.log(val), (error) => console.log(error));

quick


undefined

### And last but by no means least
## When will then() be called???

In [2]:
function RetrieveData() {
    Promise.resolve("immediate").then((val) => console.log(val));
    console.log("After Promise.then()");
}

RetrieveData();
console.log("After RetrieveData()");

After Promise.then()
After RetrieveData()


undefined

immediate


### As <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Guide/Using_promises#guarantees" target="_blank">MDN states</a>:
* *Callbacks added with then() **will never be invoked before** the completion of the **current run** of the JavaScript **event loop**.*
* *These callbacks **will be invoked** even if they were **added after the success or failure** of the asynchronous operation that the promise represents.*
* ***Multiple callbacks** may be added by calling **then() several** times. They will be invoked one after another, **in the order** in which they were **inserted**.*